# Cartpole Example
This notebook goes over the process of setting up and solving a trajectory optimization problem using multiple solvers.

## Loading the Required Packages
To set up the problem, we import `TrajectoryOptimization`, along with `StaticArrays` and `LinearAlgebra` to help in the set up.

To keep things simple, we use the cartpole model from the [`RobotZoo`](https://github.com/bjack205/RobotZoo.jl) package.

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
using TrajectoryOptimization
using RobotDynamics
import RobotZoo.Cartpole
using StaticArrays, LinearAlgebra

 Activating environment at `~/.julia/dev/TrajectoryOptimization/examples/Project.toml`
┌ Info: Precompiling RobotZoo [74be38bb-dcc2-4b9e-baf3-d6373cd95f10]
└ @ Base loading.jl:1260


### Load the model
Here we instantiate our model, which is a sub-type of `AbstractModel`, and extract out the state dimension `n` and control dimension `m`.

In [2]:
model = Cartpole()
n,m = size(model);

### Define the model discretization
We now define how we will discretize our model time. Here we choose to solve a problem with 5 second horizon and use 101 knotpoints, for a uniform time step of 0.05 seconds.

In [3]:
N = 101
tf = 5.
dt = tf/(N-1)

0.05

### Set initial and final state
All trajectory optimization problems require an initial condition (it will be assumed to be the origin if not passed to the problem constructor). The final state is not be required, and could be replaced with constraints on the terminal state, in practice. Here we want the cartpole to start with the pendulum hanging down and then swing up. Note that we use `SVector`s here, but the constructor will automatically convert normal vectors, as well.

In [4]:
x0 = @SVector zeros(n)
xf = @SVector [0, pi, 0, 0];  # i.e. swing up

### Define the Objective
Here we define a simple LQR tracking objective:
$$ \frac{1}{2} (x_N - x_f)^T Q_f (x_N - x_f) +  \frac{1}{2} \sum_{k=0}^{N-1} (x_k - x_f)^T Q_k (x_k - x_f) + (u_k - u_f)^T R_k (u_k - u_f) $$

Here it is very important to use the `Diagonal` and `SVector` constructors, since this will have a significant impact on performance. To create the objective, we use the convenient `LQRObjective` method.

In [5]:
# Set up
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = LQRObjective(Q,R,Qf,xf,N);

### Define Constraints
We now add constraints to our problem. We place simple bounds on the controls and make the goal state and hard constraint. Note that we first create an empty `ConstraintList`, then create the individual constraints, and finally add the constraints to the list, specifying the knot point indicies to which the constraint applies. Since we don't have controls at the final knot point, we set the indices to `1:N-1`.

In [13]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Control Bounds
u_bnd = 3.0
bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
add_constraint!(conSet, bnd, 1:N-1)

# Goal Constraint
goal = GoalConstraint(xf)
add_constraint!(conSet, goal, N)

### Define the problem
We now pass all the required information to the `Problem` constructor to define our problem

In [14]:
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet);

### Initialization
As is standard for all non-convex optimization problems, initialization is very important. If a good guess isn't available, it is usually best to initialize the system will a set of controls that keep the state bounded or constant. It is often useful to perturb the system slightly, and provide a dynamicall feasible state trajectory. Here we initialize the system will near-zero controls and then simulate the system forward with the `rollout!` method to provide the initial state guess.

In [15]:
u0 = @SVector fill(0.01,m)
U0 = [u0 for k = 1:N-1]
initial_controls!(prob, U0)
rollout!(prob);

## Solving with ALTRO
Now that the problem is defined, we can use any of the supported solvers to solve the problem. Here we use [`Altro.jl`](https://github.com/RoboticExplorationLab/ALTRO.jl), a very fast specialized solver that uses iterative LQR (iLQR) within an augmented Lagrangrian framework to handle the constraints.

In [16]:
using Altro
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)

altro = ALTROSolver(prob, opts)
set_options!(altro, show_summary=true)
solve!(altro);


SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 40
    Solve Time: 12.698675999999999 (ms)

  Covergence
    Terminal Cost: 1.552558743680986
    Terminal dJ: 0.0007412366246843938
    Terminal gradient: 0.003248506450786873
    Terminal constraint violation: 3.3999318915789445e-9
    Solve Status: SOLVE_SUCCEEDED


We can retrive some basic information about the solve for post-analysis,

In [17]:
println("max_violation: ", max_violation(altro))
println("cost:          ", cost(altro))
println("iterations:    ", iterations(altro));

max_violation: 3.3999318915789445e-9
cost:          1.552558743680986
iterations:    40


and extract the solution from either `prob` or `altro` using `states` and `controls`.

In [18]:
# Extract the solution
X = states(altro)
U = controls(altro)

100-element Array{SArray{Tuple{1},Float64,1,1},1}:
 [-0.053404899938501706]
 [0.6325057323965376]
 [1.2322336398580338]
 [1.7121574166104716]
 [2.0447229731492684]
 [2.2096269379689777]
 [2.195634660099169]
 [2.0032216281222563]
 [1.6467798134436662]
 [1.154168025996269]
 [0.5622664906350295]
 [-0.09019963907039578]
 [-0.7690009035661038]
 ⋮
 [2.3242657214734757]
 [2.10770503727337]
 [1.8037656724994713]
 [1.455637884059381]
 [1.0817821499545335]
 [0.6836886129973376]
 [0.2516431323969652]
 [-0.23177751883430017]
 [-0.7901242106125287]
 [-1.4532878108851093]
 [-2.2592486680672152]
 [-3.000000000027063]

These functions return vectors of vectors. If you want to convert them to 2D arrays, simply use `hcat`. If `n`, `m`, or `N` are large, it's usually a good idea to convert it to a standard Julia `Vector` before concatenation.

In [19]:
hcat(Vector.(X)...)

4×101 Array{Float64,2}:
 -3.05151e-11  -6.67562e-5    0.000591126  …  -0.00375642   8.87524e-11
  8.12449e-11   0.000133512  -0.00118987       3.13402      3.14159
 -5.4539e-11   -0.00266588    0.0289336        0.150251    -2.19716e-10
 -2.84508e-11   0.0052881    -0.0576532        0.303027     7.06432e-10

## Solving with iLQR
If we want to solve the unconstrained problem with iLQR, we simply reset our initial guess and create a new solver:

In [23]:
ilqr = Altro.iLQRSolver(prob, opts)
initial_controls!(ilqr, U0)
solve!(ilqr);


SOLVE COMPLETED
 solved using the iLQR Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 84
    Solve Time: 37.349365 (ms)

  Covergence
    Terminal Cost: 1.4497436179031664
    Terminal dJ: 6.889787558717053e-5
    Terminal gradient: 0.038402688096996665
    Solve Status: SOLVE_SUCCEEDED


## Solving with Ipopt
TrajectoryOptimization provides a way to represent a trajectory optimization as a generic nonlinear program (NLP) and solve using solvers that implement the NLP interface in [`MathOptInterface`](https://github.com/jump-dev/MathOptInterface.jl), such as Ipopt (via [`Ipopt.jl`](https://github.com/jump-dev/Ipopt.jl)). 

Before converting to an NLP, we need to explicitly add the dynamics and initial condition as constraints to the optimization problem, which we do via `add_dynamics_constraints!`.

We then convert our `Problem` to a `TrajOptNLP`, converting any applicable constraints to simple bounds on the decision variables (e.g. bound constraints and goal constraints), and specifying that the the constraint Jacobian uses a vector of non-zero values rather than a generic sparse matrix.

In [24]:
using Ipopt
using MathOptInterface
const MOI = MathOptInterface

# Copy problem to avoid modifying the original problem
prob_nlp = copy(prob)

# Add the dynamics and initial conditions as explicit constraints
TrajectoryOptimization.add_dynamics_constraints!(prob_nlp)

# Reset our initial guess
initial_controls!(prob_nlp, U0)
rollout!(prob_nlp)

# Create the NLP
nlp = TrajOptNLP(prob_nlp, remove_bounds=true, jac_type=:vector);

MethodError: MethodError: no method matching num_vars(::Int64, ::Int64, ::Int64, ::Bool)

We can now solve with Ipopt by creating the `MathOptInterface.Optimizer` and setting up the problem in the `MathOptInterface` format using `TrajectoryOptimization.build_MOI!` method

In [15]:
optimizer = Ipopt.Optimizer()
TrajectoryOptimization.build_MOI!(nlp, optimizer)
MOI.optimize!(optimizer)
MOI.get(optimizer, MOI.TerminationStatus())


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3568
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      496
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      100
                     variables with only upper bounds:        0
Total number of equa

LOCALLY_SOLVED::TerminationStatusCode = 4

Since the solve updates the data structures in `nlp` during the course of the solve, we can use either `optimizer` or `nlp` to get information after the solve.

In [16]:
println("max_violation: ", max_violation(nlp))
println("cost:          ", cost(nlp));

max_violation: 1.2434497875801753e-13
cost:          1.4958739082433778


## Comparing Solve Times
Here we demonstrate the compuational advantage of ALTRO over Ipopt. We demonstrate how to use `BenchmarkTools` to time a solve using `MathOptInterface` by resetting the initial guess between solves. As shown, ALTRO is about 100x faster than Ipopt.

In [17]:
using BenchmarkTools

# Reset initial guess and then benchmark ALTRO solver
initial_controls!(altro, U0)
b_altro = benchmark_solve!(altro)

BenchmarkTools.Trial: 
  memory estimate:  1.52 MiB
  allocs estimate:  1581
  --------------
  minimum time:     6.547 ms (0.00% GC)
  median time:      7.132 ms (0.00% GC)
  mean time:        7.095 ms (1.28% GC)
  maximum time:     7.588 ms (4.16% GC)
  --------------
  samples:          10
  evals/sample:     10

In [18]:
# Reset initial guess and benchmark Ipopt solver
initial_controls!(prob_nlp, U0)
rollout!(prob_nlp)
nlp = TrajOptNLP(prob_nlp, remove_bounds=true, jac_type=:vector)
Z0 = copy(TrajectoryOptimization.get_trajectory(nlp).Z)

optimizer = Ipopt.Optimizer(print_level=0)
TrajectoryOptimization.build_MOI!(nlp, optimizer)
Z = MOI.VariableIndex.(1:length(Z0))
b_ipopt = @benchmark begin
    MOI.optimize!($optimizer)
    MOI.set($optimizer, MOI.VariablePrimalStart(), $Z, $Z0)
    MOI.get($optimizer, MOI.TerminationStatus())
end

BenchmarkTools.Trial: 
  memory estimate:  7.79 MiB
  allocs estimate:  22123
  --------------
  minimum time:     619.169 ms (0.00% GC)
  median time:      693.174 ms (0.00% GC)
  mean time:        752.114 ms (0.08% GC)
  maximum time:     1.220 s (0.00% GC)
  --------------
  samples:          7
  evals/sample:     1

In [19]:
using Statistics
@show cost(nlp)
@show cost(altro)
@show max_violation(nlp)
@show max_violation(altro)
jdg = judge(median(b_altro), median(b_ipopt))
println("Speed improvement: ", round(1/ratio(jdg).time), "x")
jdg

cost(nlp) = 1.4958739082433778
cost(altro) = 1.5525585360226632
max_violation(nlp) = 1.2434497875801753e-13
max_violation(altro) = 3.4246698810136422e-9
Speed improvement: 97.0x


BenchmarkTools.TrialJudgement: 
  time:   -98.97% => improvement (5.00% tolerance)
  memory: -80.46% => improvement (1.00% tolerance)


## Visualizing the Results

In [20]:
using TrajOptPlots
using MeshCat
using Plots

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8702
└ @ MeshCat /home/bjack205/.julia/packages/MeshCat/ECbzr/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set(Any[]), ip"127.0.0.1", 8702))

We can use the default geometries for the robot, as defined in `TrajOptPlots`, using the `set_mesh!` command

In [21]:
TrajOptPlots.set_mesh!(vis, model)

MeshCat Visualizer with path /meshcat/robot/cart/pole at http://localhost:8702

We can visualize the results of a solver by passing the solver to `visualize!`. In practice, all a solver needs to implement for this method is a `get_model` method that returns an `AbstractModel` and a `get_trajectory` method that returns an `AbstractTrajectory`.

In [22]:
visualize!(vis, altro);

In [23]:
visualize!(vis, model, TrajectoryOptimization.get_trajectory(nlp));

We can also compare both solutions by passing in both solvers:

In [24]:
visualize!(vis, altro, nlp);

Or the model and different trajectories:

In [25]:
visualize!(vis, model, get_trajectory(altro), get_trajectory(nlp));

After this, we'll have extra geometries floating around, which can be deleting using:

In [26]:
delete!(vis["robot_copies"]);

### Tip: Defining the visualazation method
Visualization of the model is defined by the `visualize!(vis, model::MyModel, x::AbstractVector)` command:
 
 ```julia
 function TrajOptPlots.visualize!(vis, model::RobotZoo.Cartpole, x::AbstractVector)
    y = x[1]
    θ = x[2]
    q = expm((pi-θ) * @SVector [1,0,0])
    settransform!(vis["robot","cart"], Translation(0,-y,0))
    settransform!(vis["robot","cart","pole","geom"], LinearMap(UnitQuaternion(q)))
end

 ```